In [1]:
import sys
sys.path.append("..") 

## How did we construct the ground truth? 
![Ground truth](../resources/ground_truth_data_setup.png)
[edit image](https://app.diagrams.net/#G1_CWj_Ux45Q6IAAnGwVurQxSnTkOvlVG_)

# Build Ground Truth

#### Step-1: NMVW Data Dump

In [ ]:
from nmvwdatadump.data_dump import run as dump, count_all_constituents, count_all_constituent_with_wikidata

dump("ccrdfconst", 58000, range=20)
count_all_constituents("ccrdfconst") 
count_all_constituent_with_wikidata("ccrdfconst") 

#### Step-2: Filter wikidata constituents

In [ ]:
from nmvwdatadump.filter_wiki_human import run as filter_wikidata, count_total_wikidata

filter_wikidata(directory="nmvw_data/ccrdfconst") # pass the FOLDER NAME containing ttl file
count_total_wikidata("nmvw_data/ccrdfconst/const_wiki_filter_log.csv")

#### Step-3: Build ground truth

We have 6165 instance of human entities, where we know the corresponding wikidata URI, therefore consider those human entities as our ground truth. From federated query to wikidata SPARQL endpoint, we also retrieved all the naming variation(different language, spelling and format) for corresponding to 6178 human listed on wikidata in different language.
```SPARQL
select * where {
    SERVICE <https://query.wikidata.org/sparql> {  
         <"""+var+"""> rdfs:label ?name .  
    }}
```

In [ ]:
from exp100.construct_ground_truth import run as constructgroundtruthfromwikidata

constructgroundtruthfromwikidata("data/wikidata_ccrdfconstQ5_full.ttl", "data/ground_truth.pkl")

## Naive String Matching

In [ ]:
from naive.naive_string_matching import run as naive_string_matching
from utils.calculate_result import result
import pandas

naive_string_matching("data/wikidata_human_name.pkl", "results/naive_string_matching.pkl")
result("results/naive_string_matching.pkl")

In [ ]:
# show results
df = pandas.read_pickle('results/naive_string_matching.pkl')
print(df[df['retrieved_uri'].apply(lambda x: len(x)) > 0])

## Deezy Match

In [5]:
from deezymatch.deezy_match_data_construction import construct_deezymatch_data

construct_deezymatch_data('data/ground_truth.pkl', 'data/deezymatch/')

For k-fold-validation on the ground truth execute file: [k_fold_validation.ipynb](k_fold_validation/k_fold_validation.ipynb)